In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import neural_tape_controller
import optimize
import pickle
from PIL import Image, ImageDraw,ImageFont
from matplotlib import pyplot as plt
#Положительные числа - положительные награды.
root = '../'
root_dir = root

In [2]:
available_w = np.array([-1,-0.5,-0.25,0,0.25,0.5,1])
available_dv = np.array([-1,-0.5,-0.25,0,0.25,0.5,1])

In [3]:
class car():
  def __init__(self,x,y,walls):
    self.x=x
    self.y=y
    self.angle=0
    self.vx=0
    self.vy=0
    #мощности двигателя и поворота
    self.dv=0.5#макисмальная мощность
    self.dv_part=0
    self.wdv=10*3.14/180#максимальная мощность
    self.k_fr=0.12
    self.walls = walls#указатель на лабиринт
    #углы обзора
    self.view_angles = np.array([-47,-20,-8,0,8,20,47])*3.14/180
  def phis(self):
    #такт работы физики
    #работа мотора
    self.vx+= np.cos(self.angle)*self.dv_part*self.dv
    self.vy+= np.sin(self.angle)*self.dv_part*self.dv
    #кинематика
    self.vx*=1-self.k_fr
    self.vy*=1-self.k_fr
    if np.abs(self.vx)<0.01:
      self.vx=0
    if np.abs(self.vy)<0.01:
      self.vy=0
    #контакт со стеной
    part = 20
    for i in range(part):
      self.x += self.vx/part
      self.y += self.vy/part
      if self.walls.mx[int(self.x+0.5),int(self.y+0.5)]==1:
        #hit
        self.x -= self.vx/part
        self.y -= self.vy/part
        self.vx = 0
        self.vy = 0
        break
  def see(self):
    #срабатывание дальномеров
    sensor_input = []
    sensor_input_cur = 0
    i=0
    t_fract = 5
    for view_angle in self.view_angles:
      view_angle += self.angle
      delta_size = 0.1
      dx = np.cos(view_angle)*delta_size
      dy = np.sin(view_angle)*delta_size
      x = self.x
      y = self.y
      for t in range(70):
        x+=dx
        y+=dy
        sensor_input_cur=t
        if self.walls.mx[int(x+0.5),int(y+0.5)]==1:
          #hit
          break
      sensor_input.append(int(t/t_fract))
      i+=1
    return sensor_input

class maze():
  def __init__(self,size,densinty=0.5,seed=1):
    self.size = size
    self.seed = seed
    np.random.seed(seed)
    self.mx = np.random.rand(size,size)<densinty
    self.mx[0,:] = 1
    self.mx[-1,:] = 1
    self.mx[:,0] = 1
    self.mx[:,-1] = 1
    self.car=car(size/2,size/2,self)
    self.mx[int(size/2)-2:int(size/2)+2,int(size/2)-2:int(size/2)+2]=0
  def draw(self,additional_points=None):
    #additional_points = [{'color':'red','sz':0.5, 'points':[(1,2),(1,3),(2,6)]}]
    plt.axes()
    scale = 1/self.size
    for x in range(self.size):
      for y in range(self.size):
        if self.mx[x,y]:
          rectangle = plt.Rectangle(((x-0.5)*scale,(y-0.5)*scale), 1*scale, 1*scale, fc='black',ec="black")
          plt.gca().add_patch(rectangle)
    delta_size = 0.4
    dx = np.cos(self.car.angle)*delta_size
    dy = np.sin(self.car.angle)*delta_size
    rectangle = plt.Rectangle(((self.car.x-dx)*scale,(self.car.y-dy)*scale), 1*scale, 1*scale, fc='blue')
    plt.gca().add_patch(rectangle)
    rectangle = plt.Rectangle((self.car.x*scale,self.car.y*scale), 1*scale, 1*scale, fc='red')
    plt.gca().add_patch(rectangle)
    
    if not (additional_points is None):
        for additional_point in additional_points:
            color = additional_point['color']
            sz = additional_point['sz']
            for point in additional_point['points']:
                #print('point',point,'color',color)
                rectangle = plt.Rectangle((point[0]*scale,point[1]*scale), sz*scale, sz*scale, fc=color)
                plt.gca().add_patch(rectangle)
    plt.show()

In [4]:
from PIL import Image, ImageDraw,ImageFont
def make_trajectory(m,ln,controller,goal=None,get_traj=False,draw=False):
  if goal==None:
      tx = int(np.random.rand()*(np.shape(m.mx)[0]-1))
      ty = int(np.random.rand()*(np.shape(m.mx)[1]-1))
  else:
    tx = goal[0]
    ty = goal[1]
    
  traj_coords = []  
  prev_x = 0
  prev_y = 0
  log = []
  m.car.angle = np.random.rand()*10
  reward_lst = []
  for t in range(ln):
    
    state = m.car.see() + [tx,ty]
    [a_w_num,a_dv_num] = controller.act(state,0,0)
    
    t_range = np.abs(state[-2]-state[-4])+np.abs(state[-1]-state[-3])
    reward = 1/(t_range**2+0.0001)
    if reward>1:
        reward = 1
    reward_lst.append(reward)
    
    if a_w_num<0:
        a_w_num=0
    if a_w_num>6:
        a_w_num=6
    if a_dv_num<0:
        a_dv_num=0
    if a_dv_num>6:
        a_dv_num=6
    a_dv_num = int(a_dv_num)
    a_w_num = int(a_w_num)
    log.append(a_w_num)#1
    log.append(a_dv_num)#1
    log.extend(m.car.see())#7
    if (prev_x==0 and prev_y==0) or ((np.abs(prev_x-m.car.x)<5) and (np.abs(prev_y-m.car.y)<5)):
      prev_x=m.car.x
      prev_y=m.car.y
      traj_coords.append((m.car.x,m.car.y))
    else:
      print('РАЗРЫВ ТРАЕКТОРИИ')
      print('prev_x,prev_y',prev_x,prev_y,'m.car.x,m.car.y',m.car.x,m.car.y)
      print(log)
      1/0


    log.append(int(m.car.x))#1
    log.append(int(m.car.y))#1
    w=available_w[a_w_num]
    dv=available_dv[a_dv_num]

    m.car.dv_part=dv
    m.car.angle += m.car.wdv*w
    m.car.phis()
    
    size = m.size
    if draw:
        scale = 8
        im = Image.new('RGB', (size*scale, size*scale), (256, 256, 256))
        dr = ImageDraw.Draw(im)
        for x in range(size):
          for y in range(size):
            if m.mx[x,y]:
              color = 'black'
              shape = ((x-0.5)*scale,(y-0.5)*scale, (x+0.5)*scale, (y+0.5)*scale)
              dr.rectangle(shape, fill=color)
                
        delta_size = 0.5
        dx = np.cos(m.car.angle)*delta_size
        dy = np.sin(m.car.angle)*delta_size
        shape = ((m.car.x-dx-0.5)*scale,(m.car.y-dy-0.5)*scale, (m.car.x-dx+0.5)*scale, (m.car.y-dy+0.5)*scale)
        dr.rectangle(shape, fill='blue')
        shape = ((m.car.x-0.5)*scale,(m.car.y-0.5)*scale, (m.car.x+0.5)*scale, (m.car.y+0.5)*scale)
        dr.rectangle(shape, fill='red')
        
        #goal
        shape = ((tx+0.5)*scale,(ty+0.5)*scale, (tx-0.5)*scale, (ty-0.5)*scale)
        dr.rectangle(shape, fill='lime')
        globals()['video'].append(im)
        
        #plt.imshow(im)
        #plt.show()
  return np.sum(reward_lst)

In [5]:
state_dim = 9
n_actions = 2

In [6]:
def check_genom(genom,draw=False):
  if draw:
      globals()['video'] = []
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  ln = 150
  q_arr = []
  i = 0
  for i in range(20):
    m = maze(30,seed=i,densinty=0.4)
    q = make_trajectory(m,ln,controller,draw=draw)
    q -= np.sum(genom**2)*0.000001#регуляризация
    q_arr.append(q)
  if draw:
      print(q_arr)
  return (np.mean(q_arr) + np.min(q_arr))*0.5#взвешенное среднее, с повышенным весом за провалы

In [7]:
nt = neural_tape_controller.nt_controller(input_size=state_dim,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [8]:
if 0:
    with open(root_dir+f'genoms/best_genom_maze.pkl', 'rb') as f:
        genoms = pickle.load(f)
        genom = genoms[-1]

In [9]:
%%time
q = check_genom(genom,draw=True)
print(q)

in_ar [[ 0  0  3  3  3  3  3  4 13 23 10]]
inp [ 0  0  3  3  3  3  3  4 13 23 10]
state [[ 3  3  3  3  3  4 13 23 10]]
[reward,done] [0, 0]


ZeroDivisionError: division by zero

In [10]:
if 0:
    _video = globals()['video']
    _video[0].save(
      root+'out_videos/maze_neuroevol.gif',
      save_all=True,
      append_images=_video[1:], 
      optimize=True,
      duration=100,
      loop=0
    )

In [11]:
pd.Timestamp.now()

Timestamp('2022-05-11 13:11:15.552646')

In [12]:
print(1)
opt = optimize.optimizer(check_genom, genom_size=len(genom),parallel_cores=1)
try:
    with open(root_dir+f'genoms/best_genom_maze.pkl', 'rb') as f:
        genoms = pickle.load(f)
        genom = genoms[-1]
        opt.best_genoms = genoms
    print('loaded successfully')
except Exception:
    nt = neural_tape_controller.nt_controller(input_size=state_dim,output_size=n_actions)
    genom = nt.nn.disassemble_genom()

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root+f'genoms/best_genom_maze.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

1
in_ar [[ 0  0  3  3  3  3  3  4 13 23 10]]
inp [ 0  0  3  3  3  3  3  4 13 23 10]
state [[ 3  3  3  3  3  4 13 23 10]]
[reward,done] [0, 0]


ZeroDivisionError: division by zero